
# Thermal Person Detector — YOLOv8 (Clean, Portable Notebook)
**KR**: 이 노트북은 열화상 이미지에서 사람을 탐지하는 YOLOv8 워크플로우를 재현 가능한 형태로 제공합니다.  
Colab 전용이 아니라 로컬(Windows/macOS/Linux) 또는 다른 환경에서도 그대로 실행할 수 있도록 구성했습니다.

**EN**: This notebook provides a reproducible YOLOv8 workflow for detecting people in thermal images.  
It is designed to run anywhere (local Windows/macOS/Linux or other environments), not just on Colab.



## Contents / 목차
1. Environment Setup (로컬 실행 준비)  
2. Project Structure & Paths (프로젝트 구조와 경로)  
3. (Optional) Dataset Preparation (선택) 데이터셋 준비  
4. Quick Inference with Existing Weights (가중치로 바로 추론)  
5. (Optional) Training (선택) 학습  
6. Export to ONNX (ONNX로 내보내기)  
7. ONNX Sanity Check (ONNX 무결성/드라이런 확인)  
8. (Optional) TFLite Conversion (선택) TFLite 변환



## 1) Environment Setup / 로컬 실행 준비
**KR**: 아래 셀은 필수 패키지를 설치하고, 환경 정보를 확인합니다.  
**EN**: The cell below installs required packages and prints environment info.


In [ ]:

# If you're on a managed environment, you may skip installs that are already satisfied.
# 필요시만 설치하세요. (Ultralytics, ONNX, ONNXRuntime)
import sys, subprocess

def pip_install(package):
    try:
        __import__(package.split("==")[0].replace("-", "_"))
    except Exception:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Minimal dependencies (pin lightly if needed)
for pkg in ["ultralytics>=8.2.0", "onnx>=1.15.0", "onnxruntime>=1.17.0", "opencv-python>=4.7.0", "numpy>=1.26.0"]:
    pip_install(pkg)

import platform, torch, onnx, onnxruntime as ort, cv2, numpy as np
import ultralytics
print("Python:", sys.version)
print("OS:", platform.platform())
print("Torch:", torch.__version__, "CUDA available?", torch.cuda.is_available())
print("Ultralytics:", ultralytics.__version__)
print("OpenCV:", cv2.__version__)
print("ONNX:", onnx.__version__)
print("ONNXRUNTIME:", ort.__version__)



## 2) Project Structure & Paths / 프로젝트 구조와 경로
**KR**: 프로젝트 루트와 입출력 경로를 지정합니다. OS에 관계없이 동작하도록 `os.path`를 사용합니다.  
**EN**: Define root and I/O paths. We use `os.path` to be OS-agnostic.


In [ ]:

import os

# You can change ROOT to where your project lives
ROOT = os.path.abspath(".")
DATA_DIR = os.path.join(ROOT, "data")          # your dataset root (images/, labels/)
WEIGHTS_DIR = os.path.join(ROOT, "weights")    # store trained / downloaded weights
EXPORT_DIR = os.path.join(ROOT, "export")      # exported ONNX etc.
RESULTS_DIR = os.path.join(ROOT, "results")    # inference outputs

for d in [DATA_DIR, WEIGHTS_DIR, EXPORT_DIR, RESULTS_DIR]:
    os.makedirs(d, exist_ok=True)

print("ROOT:", ROOT)
print("DATA_DIR:", DATA_DIR)
print("WEIGHTS_DIR:", WEIGHTS_DIR)
print("EXPORT_DIR:", EXPORT_DIR)
print("RESULTS_DIR:", RESULTS_DIR)



## 3) (Optional) Dataset Preparation / (선택) 데이터셋 준비
**KR**: YOLO 형식의 데이터셋(`images/train|val`, `labels/train|val`, `.yaml`)을 사용합니다.  
이미 준비된 `.yaml`이 있다면, 경로만 맞추면 됩니다.  
**EN**: We use a standard YOLO dataset layout. If you already have a `.yaml`, set correct paths.


In [ ]:

# Example YAML template generator (edit paths/classes as needed)
yaml_path = os.path.join(DATA_DIR, "thermal_person.yaml")
yaml_text = f"""
# YOLO dataset YAML for thermal person detection
path: {DATA_DIR}
train: images/train
val: images/val
test: images/test

names:
  0: person
"""
with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(yaml_text)

print("Wrote dataset YAML ->", yaml_path)
print("Edit it as needed to match your folder structure.")



## 4) Quick Inference with Existing Weights / 가중치로 바로 추론
**KR**: 이미 학습된 가중치(.pt 또는 .engine 등)가 있다면, 아래처럼 즉시 추론을 실행합니다.  
**EN**: If you have pre-trained weights (.pt), run quick inference as below.


In [ ]:

from ultralytics import YOLO
import glob

# Point to an existing .pt (replace with your actual file)
candidate_pts = glob.glob(os.path.join(WEIGHTS_DIR, "*.pt"))
if candidate_pts:
    weights_path = candidate_pts[0]
else:
    # Download a small model if none found (you can change to 'yolov8n.pt' / 'yolov8n-seg.pt', etc.)
    weights_path = os.path.join(WEIGHTS_DIR, "yolov8n.pt")
    if not os.path.exists(weights_path):
        model = YOLO("yolov8n.pt")
        model.export(format="pt")  # this will download weights into a temp dir; we just keep using hub weights
        # Alternatively, save a local copy
        import shutil
        src = model.ckpt_path if hasattr(model, "ckpt_path") else None
        if src and os.path.exists(src):
            shutil.copy(src, weights_path)

print("Using weights:", weights_path)

# Input image(s)
SAMPLE_IMG = os.path.join(DATA_DIR, "sample.jpg")  # replace with your thermal image
if not os.path.exists(SAMPLE_IMG):
    # Create a dummy image if missing
    import numpy as np, cv2
    dummy = np.zeros((256,256,3), dtype=np.uint8)
    cv2.putText(dummy, "Place thermal image here", (10,130), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)
    cv2.imwrite(SAMPLE_IMG, dummy)

# Run inference
model = YOLO(weights_path)
res = model.predict(source=SAMPLE_IMG, save=True, project=RESULTS_DIR, name="infer_once", imgsz=640, conf=0.25)
print("Result dir:", os.path.join(RESULTS_DIR, "infer_once"))



## 5) (Optional) Training / (선택) 학습
**KR**: 학습은 데이터셋이 준비되어 있을 때만 실행하세요. 빠른 데모를 위해 작은 epoch로 설정되어 있습니다.  
**EN**: Run training only if your dataset is ready. We keep epochs small for a quick demo.


In [ ]:

# Train a small model (edit hyperparameters as needed)
# NOTE: You must have DATA_DIR/images/train & labels/train ready.
do_train = False  # set True to train

if do_train:
    model = YOLO("yolov8n.pt")  # or a custom base
    model.train(
        data=os.path.join(DATA_DIR, "thermal_person.yaml"),
        epochs=5,
        imgsz=640,
        batch=8,
        device=0 if torch.cuda.is_available() else "cpu",
        project=os.path.join(ROOT, "runs"),
        name="thermal_yolov8_demo",
        seed=42
    )
    # Save trained weights to WEIGHTS_DIR
    last = os.path.join(ROOT, "runs", "detect", "thermal_yolov8_demo", "weights", "last.pt")
    best = os.path.join(ROOT, "runs", "detect", "thermal_yolov8_demo", "weights", "best.pt")
    if os.path.exists(best):
        import shutil
        shutil.copy(best, os.path.join(WEIGHTS_DIR, "best.pt"))
        print("Copied best.pt ->", os.path.join(WEIGHTS_DIR, "best.pt"))
    elif os.path.exists(last):
        shutil.copy(last, os.path.join(WEIGHTS_DIR, "last.pt"))
        print("Copied last.pt ->", os.path.join(WEIGHTS_DIR, "last.pt"))
else:
    print("Training skipped. Set do_train=True to enable training.")



## 6) Export to ONNX / ONNX로 내보내기
**KR**: 추론용 가중치(.pt)를 ONNX로 변환합니다. opset, dynamic batch 등의 옵션을 노출했습니다.  
**EN**: Convert .pt weights to ONNX. We expose opset and dynamic options commonly needed.


In [ ]:

from ultralytics import YOLO
import os

# Choose which weights to export (existing or just used above)
export_src = candidate_pts[0] if 'candidate_pts' in globals() and candidate_pts else os.path.join(WEIGHTS_DIR, "yolov8n.pt")
print("Export source:", export_src)

model = YOLO(export_src)
onnx_out = os.path.join(EXPORT_DIR, "model.onnx")
model.export(format="onnx", opset=12, dynamic=True, imgsz=640, half=False)  # Ultralytics writes to its own path

# Find the exported ONNX (Ultralytics typically names it automatically in the working dir)
# We'll search for the newest .onnx under EXPORT_DIR or ROOT
import glob, time
candidates = glob.glob(os.path.join(ROOT, "**", "*.onnx"), recursive=True)
candidates = sorted(candidates, key=lambda p: os.path.getmtime(p), reverse=True)
if candidates:
    latest_onnx = candidates[0]
    # Copy to EXPORT_DIR/model.onnx
    import shutil
    shutil.copy(latest_onnx, onnx_out)
    print("Copied ONNX ->", onnx_out)
else:
    print("No ONNX found. Check export logs.")



## 7) ONNX Sanity Check / ONNX 무결성 확인 & 드라이런
**KR**: ONNX 모델을 로드해 shape를 확인하고, 임의 입력으로 드라이런합니다.  
**EN**: Load the ONNX model, inspect I/O shapes, and run a dry inference with random data.


In [ ]:

import onnx
import onnxruntime as ort
import numpy as np, os, cv2

onnx_path = os.path.join(EXPORT_DIR, "model.onnx")
assert os.path.exists(onnx_path), f"ONNX not found: {onnx_path}"

model = onnx.load(onnx_path)
onnx.checker.check_model(model)
print("ONNX checked OK")

sess = ort.InferenceSession(onnx_path, providers=['CPUExecutionProvider'])
inputs = sess.get_inputs()
outputs = sess.get_outputs()

print("Inputs:", [(i.name, i.shape, i.type) for i in inputs])
print("Outputs:", [(o.name, o.shape, o.type) for o in outputs])

# Make a dummy input that matches the expected shape (usually [1,3,640,640])
# If dynamic, convert SAMPLE_IMG to 640x640 and feed as NCHW.
img = cv2.imread(SAMPLE_IMG, cv2.IMREAD_COLOR)
img = cv2.resize(img, (640, 640), interpolation=cv2.INTER_LINEAR)
x = img[:, :, ::-1].transpose(2,0,1).astype(np.float32) / 255.0  # BGR->RGB, HWC->CHW, [0,1]
x = np.expand_dims(x, 0)  # [1,3,640,640]

feed = {inputs[0].name: x}
pred = sess.run([o.name for o in outputs], feed)
print("ONNX forward OK. Got", len(pred), "output(s).")



## 8) (Optional) TFLite Conversion / (선택) TFLite 변환
**KR**: 아래는 일반적인 PyTorch→ONNX→TFLite 경로가 아닌, Ultralytics의 SavedModel 경유 또는 다른 툴체인을 사용하는 흐름이 필요할 수 있습니다.  
프로젝트 요구사항에 따라 변환 툴과 버전을 엄격히 맞추세요.  
**EN**: For TFLite, you may need different toolchains or export routes (e.g., SavedModel).  
Match the tool versions precisely per your deployment target.


In [ ]:

# Placeholder for TFLite conversion (highly version-sensitive; provide your own pipeline as needed).
print("TFLite conversion is environment-specific. Add your converter steps here if required.")



---

### Notes / 비고
- **Data & Licenses**: If you're using subsets of public datasets (e.g., Set-A only), clearly state it in your README and follow licenses.  
- **Reproducibility**: Pin versions (requirements.txt) and seeds where necessary. Document opset/imgsz/conf/iou/NMS, etc.  
- **Portability**: This notebook avoids Colab-only paths/commands. It should run on most machines with Python≥3.9.

**Happy building!**
